In [1]:
import sys
import math
import numpy as np
sys.path.insert(0, '..')
from net_framework import *
from sklearn.model_selection import train_test_split
import pandas as pd
import matplotlib.pyplot as plt
import random
import json

In [2]:
term_data = pd.read_csv('term.txt', sep="\t", header=None)
term_data.columns = ["#Lnum", "#snum", "#cnum", "Term Abbrev"]
term_data.head()

,#Lnum,#snum,#cnum,Term Abbrev
0,1,1,1,LB
1,1,1,2,LB
2,1,1,3,LE
3,1,1,4,WK
4,1,1,5,LF


In [30]:
cnum_data = pd.read_csv('cnum-vhcm-lab-new.txt', sep="\t")
locations = cnum_data[['#cnum']]
locations['Normalized-L'] = (cnum_data['L*'] - cnum_data['L*'].mean())/(cnum_data['L*'] - cnum_data['L*'].mean()).max()
locations['Normalized-a'] = (cnum_data['a*'] - cnum_data['a*'].mean())/(cnum_data['a*'] - cnum_data['a*'].mean()).std() * 1/2
locations['Normalized-b'] = (cnum_data['b*'] - cnum_data['b*'].mean())/(cnum_data['b*'] - cnum_data['b*'].mean()).std() * 1/2
display(locations.head(5))

<ipython-input-30-f811cbef4bec>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  locations['Normalized-L'] = (cnum_data['L*'] - cnum_data['L*'].mean())/(cnum_data['L*'] - cnum_data['L*'].mean()).max()
<ipython-input-30-f811cbef4bec>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  locations['Normalized-a'] = (cnum_data['a*'] - cnum_data['a*'].mean())/(cnum_data['a*'] - cnum_data['a*'].mean()).std() * 1/2
<ipython-input-30-f811cbef4bec>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a s

,#cnum,Normalized-L,Normalized-a,Normalized-b
0,141,1.000000,-0.025675,-0.138822
1,274,0.876301,-0.025504,-0.138822
2,129,0.876301,0.070445,-0.106039
3,230,0.876301,0.070101,-0.089951
4,302,0.876301,0.070617,-0.072041


In [31]:
locations = locations.sort_values('#cnum')
chip_num = list(locations['#cnum'])
lab_norm = [[row[2], row[3], row[4]] for row in locations.itertuples()]

In [32]:
l1 = term_data[term_data.get('#Lnum').eq(2)]
unique_symbols = list(l1['Term Abbrev'].unique())
l1_grouped = l1.groupby('#cnum')['Term Abbrev'].apply(list)
l1_chip_abbrev_percentage = [[(l1_grouped[i + 1].count(abbrev) / len(l1_grouped[i + 1])) for abbrev in unique_symbols] for i in range(len(l1_grouped))]

In [33]:
l1_result = pd.DataFrame(l1_chip_abbrev_percentage)
l1_result.index += 2
l1_result.index.name = '#cnum'
l1_result.columns = unique_symbols
print(l1_result)
chip_norm = []
#pull the percentage for each cnum
for x in chip_num:
    chip_norm.append((l1_result.loc[l1["#cnum"]==x]).values.tolist()[0])

             YN        NR        TK        EP        KA        AA        IT  \
#cnum                                                                         
2      0.208333  0.000000  0.166667  0.000000  0.000000  0.000000  0.458333   
3      0.416667  0.000000  0.041667  0.208333  0.000000  0.041667  0.125000   
4      0.000000  1.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
5      0.000000  0.083333  0.458333  0.000000  0.000000  0.041667  0.083333   
6      0.375000  0.000000  0.041667  0.041667  0.000000  0.041667  0.458333   
...         ...       ...       ...       ...       ...       ...       ...   
327    0.000000  0.000000  0.000000  0.041667  0.000000  0.708333  0.000000   
328    0.000000  0.000000  0.041667  0.875000  0.000000  0.000000  0.041667   
329    0.583333  0.000000  0.250000  0.000000  0.000000  0.000000  0.041667   
330    0.041667  0.083333  0.208333  0.000000  0.083333  0.125000  0.000000   
331    0.083333  0.000000  0.041667  0.833333  0.000

IndexingError: Unalignable boolean Series provided as indexer (index of the boolean Series and of the indexed object do not match).

In [ ]:
node_num = range(1,25)
layer_num = range(1,4)


shape_collection = []

def trickle(arr, iteration_left, check):
    if iteration_left == 0:
        global shape_collection
        #running the int fxn to make sure we don't have floats
        mp = map(int, arr)
        x = list(mp)
        if check == sum(x):
            shape_collection.append(x)
    else:
        new_arr = [0]+ arr + [0]
        #recursively expanding the list symmetrically
        while new_arr[0] < new_arr[1]-2 and new_arr[-1] < new_arr[-2]-2:
            new_arr[0] += 1
            new_arr[1] -= 1
            new_arr[-1] += 1
            new_arr[-2] -= 1
        trickle(new_arr, iteration_left - 1, check)

for node in node_num:
    for layer in layer_num:
        if node//layer < 3:
            continue
        if layer%2 == 0:
            trickle([node/2, node/2], (layer-2)/2, node)
        else:
            trickle([node], (layer-1)/2, node)
        
        
        
        

print(shape_collection)

#Originally designed to find every single permutation
#Oops misread directions
node_num = range(10, 130, 10)
layer_num = range(1,6)

def check(arr):
    for a in arr:
        if a < 3:
            return False
    inverse = arr[::-1]
    if arr == inverse:
        return True
    else:
        return False

shape_collection = []

def trickle_down(node, layer):
    if layer > 1:
        for i in range(0, node[-1]):
            new_arr = node[:-1] + [node[-1]-i] + [i]
            trickle_down(new_arr,layer-1)
    else:
        if check(node):
            global shape_collection
            shape_collection.append([sum(node), len(node),node])

for node in node_num:
    for layer in layer_num:
        trickle_down([node], layer)

print(shape_collection)

In [ ]:
#Number of training iterations
num_iters = 500

#Listing out the shapes of each model
colors_num = len(chip_norm[0])
input_size = 3

network_shapes = []
for s in shape_collection:
    network_shapes.append((input_size,s,colors_num))

#Learning rate of the network
rate = 0.001

#Generating Training Data
input_train, output_train, input_test, output_test = 0, 0, 0, 0
def shuffle():
    lab_train, lab_test, chip_train, chip_test = train_test_split(lab_norm, chip_norm, test_size=0.2, random_state = 3, shuffle = True)
    #test run on whole dataset
    #lab_train, lab_test = lab_norm, lab_norm
    #chip_train, chip_test = chip_norm, chip_norm
    
    global input_train, output_train, input_test, output_test
    input_train = torch.FloatTensor(lab_train)
    output_train = torch.FloatTensor(chip_train)
    input_test= torch.FloatTensor(lab_test)
    output_test = torch.FloatTensor(chip_test)

print(network_shapes)

In [ ]:
#Array of losses over training period for each network
output_file = {}
for n in node_num:
    output_file[n] = {}
    
for net_num, shape in enumerate(network_shapes):
    shuffle()
    print("Training: ",shape)
    NN = Neural_Network(inputSize = shape[0], outputSize = shape[2],
                        hiddenSize = shape[1] , learning_rate = rate)
    error_arr = []
    prev_error = 0
    strike = 0
    
    for i in range(num_iters):       
        NN.train(input_train, output_train)
        validation_error = NN.l1error(output_test, NN(input_test))
        #zero mistake counter at new training
        if i == 0:
            strike = 0
        #adding error to array
        error_arr.append(validation_error)
        #wait for them to grow up
        if prev_error < validation_error and i > 100:
            if strike > 3:
                print("Complete at iteration ", i, "\nFinal error: ", validation_error, "\n")
                break
            else:
                strike += 1
        prev_error = validation_error

    #Saves the training results in a filed named "Net 0", "Net 1", etc. 
    NN.saveWeights(model = NN, path = "saved_networks/Net " + str(net_num))
    output_file[sum(shape[1])][len(shape[1])] = error_arr

In [ ]:
with open('validation_errors.json', 'w') as f:
    json.dump(output_file, f)